<a href="https://colab.research.google.com/github/Aya11ali/Shouf/blob/main/Clean_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q  --upgrade --force-reinstall gensim


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 4.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [5]:
!pip install -q pandas==2.2.2 --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 24.8 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.2
    Uninstalling pytz-2025.2:
      Successfully uninstalled pytz-2025.2
  Attempting uninstall: tzdata
    Found existing installation: tzdata 2025.2
    Uninstalling tzdata-2025.2:
      Successfully uninstalled tzdata-2025.2
  Attempting uninstall: six
    Found existing installation: six 1.17.0
    Uninstalling six-1.17.0:
      Successfully uninstalled six-1.17.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.

In [24]:
import pandas as pd
import numpy as np
from abc import ABC, abstractmethod
from typing import List, Iterable, Dict

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds

# import os
# import gensim.downloader as api
# import torch
# from gensim.models import KeyedVectors


# import nltk
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
# from sklearn.feature_extraction.text import TfidfVectorizer

# Preparing Data

The following module defines the interface and implementation for loading and shuffling a dataset from a CSV file.

It includes an abstract base class
```ILoad_and_shuffle_Dataset```
, which outlines the essential methods for loading and shuffling data, and the concrete implementation ```Load_and_shuffle_Dataset```
, which implements these methods and provides functionality for reproducible shuffling using a random seed.


In [2]:
class ILoad_and_shuffle_Dataset(ABC):
  @abstractmethod
  def load_dataset(self):
    pass

  @abstractmethod
  def shuffle_data(self):
    pass

class Load_and_shuffle_Dataset(ILoad_and_shuffle_Dataset):
  def __init__(self,dataset_path:str,random_seed:int=42):
    self.random_seed = random_seed
    np.random.seed(self.random_seed) # Set seed for NumPy's random number generator to ensure reproducible results
    self.dataset_path = dataset_path
    self.df = None
    self.load_dataset()
    self.shuffle_data()


  def load_dataset(self):
    self.df = pd.read_csv(self.dataset_path)

  def shuffle_data(self):
    self.df = self.df.sample(frac=1, random_state=self.random_seed).reset_index(drop=True)

The following module defines a structure for calculating interaction scores in a dataset based on different user actions (e.g., "watch", "like", "comment", "subscribe"). It uses a column-matching strategy to identify relevant columns, applies weights to interaction types, and computes a final score for each record.

In [3]:
class IInteractionScorer(ABC):
    @abstractmethod
    def compute_interaction_score(self):
        pass


class ColumnMatcher(ABC):
    @abstractmethod
    def match_columns(self, df_columns, interaction_type):
        pass


class SubstringMatcher(ColumnMatcher):
    def match_columns(self, df_columns, interaction_type=None):
        if interaction_type is None:
            raise ValueError("interaction_type must be provided")
        return [col for col in df_columns if interaction_type in col]


class InteractionScorer(IInteractionScorer):
    def __init__(self, dataframe, weights:dict=None, matcher: ColumnMatcher = SubstringMatcher):
        self.df = dataframe
        self.weights = weights or {
            "watch": 1,
            "like": 2,
            "comment": 3,
            "subscribe": 5
        }
        self.matcher = matcher()
        self.compute_interaction_score()

    def compute_interaction_score(self):
        interaction_score = 0
        for interaction_type, weight in self.weights.items():
            matched_cols = self.matcher.match_columns(self.df.columns, interaction_type)
            for col in matched_cols:
                interaction_score += self.df[col] * weight
        self.df["interaction_score"] = interaction_score


In [4]:
class DataPreparingManager:
  def __init__(self, loader:ILoad_and_shuffle_Dataset, dataset_path:str,score:IInteractionScorer):
    self.loader = loader(dataset_path)
    self.df = self.loader.df
    self.interaction_score = score(self.df)

  def prepare_data(self):
    return self.df

# Collaborative Filtering Recommendation System

The following  module defines an abstract interface and a concrete implementation for creating a ***User-Item interaction*** matrix from a given dataset.

`UserItemMatrix` (Concrete Implementation)


Implements `IUserItemMatrix` to generate a matrix from a DataFrame using specified rows, columns, and values.

**Constructor Parameters:**

`dataframe` (pd.DataFrame): Input dataset.

`row` (str): Column to use as index (e.g., user_id).

`column` (str): Column to use as columns (e.g., video_id).

`values` (str): Column containing interaction values (e.g., interaction_score).

In [5]:
class IUserItemMatrix(ABC):
  @abstractmethod
  def create_user_item_matrix(self)->pd.DataFrame:
    pass

class UserItemMatrix(IUserItemMatrix):
  def __init__(self, dataframe:pd.DataFrame, row:str, column:str, values:str):
    self.df = dataframe
    self.row = row
    self.column = column
    self.values = values

  def create_user_item_matrix(self)->pd.DataFrame:
    if self.row not in self.df.columns or self.column not in self.df.columns:
        raise InvalidMatrixError(f"Invalid columns: {self.row} or {self.column} not found in the dataframe.")

    return self.df.pivot(index=self.row, columns=self.column, values=self.values).fillna(0)

  @property
  def matrix(self) -> pd.DataFrame:
      return self.create_user_item_matrix()

The following  module defines an interface and implementation for converting a dense matrix into a sparse matrix format

In [31]:
class ISparseMatrixConverter(ABC):
    @abstractmethod
    def create_sparse_matrix(self)  -> csr_matrix:
        pass

class SparseMatrixConverter(ISparseMatrixConverter):
    def __init__(self, dense_matrix: pd.DataFrame):
        self.df = dense_matrix
        self._sparse_matrix = None

    def create_sparse_matrix(self)  -> csr_matrix:
        if self._sparse_matrix is None:
            self._sparse_matrix = csr_matrix(self.df.values)
        return self._sparse_matrix

    @property
    def sparse_matrix(self):
        return self.create_sparse_matrix()

The following module Trains a K-Nearest Neighbors (KNN) model on a given sparse user-item interaction matrix, supporting custom similarity metrics and algorithms for building recommender systems.

In [7]:
class IKNNModelTrainer(ABC):
    @abstractmethod
    def train_model(self) -> NearestNeighbors:
        pass

class KNNModelTrainer(IKNNModelTrainer):
    def __init__(self, sparse_matrix: csr_matrix, metric: str = "cosine",
                 algorithm: str = "brute"):
        self.sparse_matrix = sparse_matrix
        self.metric = metric
        self.algorithm = algorithm
        self._model = None

    def train_model(self) -> NearestNeighbors:
      try:
          if self._model is None:
              self._model = NearestNeighbors(metric=self.metric,
                                             algorithm=self.algorithm)
              self._model.fit(self.sparse_matrix)

      except Exception as e:
          raise TrainingError(f"Model training failed: {e}")

      return self._model

    @property
    def model(self) -> NearestNeighbors:
        return self.train_model()


The following class Validates whether a given user_id exists in the user-item interaction matrix, ensuring the user has interacted with the system and that the matrix is properly initialized.

In [8]:
# class IUserInteractionChecker(ABC):
  @abstractmethod
#   def check_user_interaction(self)->bool:
#     pass

# class UserInteractionChecker(IUserInteractionChecker):
#   def __init__ (self, user_id:int, user_item_matrix:pd.DataFrame):
#     self.user_id = user_id
#     self.user_item_matrix = user_item_matrix

#   def check_user_interaction(self) -> bool:
#       if self.user_item_matrix is None or self.user_item_matrix.empty:
#           raise InvalidMatrixError("User-Item matrix is empty or not initialized.")

#       if self.user_id not in self.user_item_matrix.index:
#           raise InvalidUserIdError(self.user_id)
#       return True


class IUserInteractionChecker(ABC):
    @abstractmethod
    def check_user_interaction(self, user_id: int, user_item_matrix: pd.DataFrame) -> bool:
        pass

class UserInteractionChecker(IUserInteractionChecker):
    def check_user_interaction(self, user_id: int, user_item_matrix: pd.DataFrame) -> bool:
        if user_item_matrix is None or user_item_matrix.empty:
            raise InvalidMatrixError("User-Item matrix is empty or not initialized.")
        if user_id not in user_item_matrix.index:
            raise InvalidUserIdError(user_id)
        return True


The following class retrieves the index position of a given `user_id` within a user-item matrix, typically used as input for recommendation models

In [9]:
class IUserIndexFinder(ABC):
    @abstractmethod
    def _find_user_index(self) -> int:
        """Finds the index of the user in the user-item matrix."""
        pass

    @property
    @abstractmethod
    def index(self):
        pass

class UserIndexFinder(IUserIndexFinder):
    def __init__(self, user_id: int, user_item_matrix: pd.DataFrame, user_check: IUserInteractionChecker):
        self.user_id = user_id
        self.user_item_matrix = user_item_matrix
        self.user_check = user_check

    def _find_user_index(self) -> int:
        # Check user interaction when needed
        self.user_check.check_user_interaction(self.user_id, self.user_item_matrix)

        return self.user_item_matrix.index.get_loc(self.user_id)

    @property
    def index(self) -> int:
        return self._find_user_index()


The following class finds and provides a list of similar users for a given `user_id`, based on a user-item matrix and a K-Nearest Neighbors (KNN) model.

In [33]:
class ISimilarUserFinder(ABC):
  @abstractmethod
  def find_similar_users(self, n_neighbors: int = 5) -> np.ndarray:
      """Returns indices of similar users for a given user ID."""
      pass

  @property
  @abstractmethod
  def similar_users(self) -> np.ndarray:
      """Returns cached similar users."""
      pass

class SimilarUserFinder(ISimilarUserFinder):
  def __init__(self, user_id: int, knn_model, user_item_matrix: pd.DataFrame,
              user_index: IUserIndexFinder, user_check: IUserInteractionChecker):
      self.user_id = user_id
      self.knn_model = knn_model
      self.user_item_matrix = user_item_matrix
      self.user_check = user_check
      self.user_index = user_index
      self._similar_users = None

      self.user_check.check_user_interaction(self.user_id, self.user_item_matrix)
      self._user_index = self.user_index.index

  def find_similar_users(self, n_neighbors: int = 5) -> np.ndarray:

      if self.knn_model is None:
          raise RecommendationSystemError("KNN model is not provided.")

      # Get the user vector
      user_vector = self.user_item_matrix.iloc[self._user_index].values.reshape(1, -1)

      # Find neighbors
      distances, indices = self.knn_model.kneighbors(user_vector,
                                                     n_neighbors=n_neighbors + 1)

      # Exclude the first index (it will be the user itself)
      self._similar_users = indices[0][1:]
      return self._similar_users

  @property
  def similar_users(self) -> np.ndarray:
    if self._similar_users is None:
        return self.find_similar_users()
    return self._similar_users

The following class provides user-based collaborative filtering recommendations. The class uses the similar users' interactions with items to recommend items to a given user, excluding items the user has already interacted with.

In [17]:
class IUserBasedRecommender(ABC):
    @abstractmethod
    def recommend(self, top_n: int = 5) -> List[int]:
        """Returns a list of item IDs recommended for the user."""
        pass

class UserBasedRecommender(IUserBasedRecommender):
    def __init__(self, user_id: int, user_item_matrix: pd.DataFrame,
                 similar_user_finder: ISimilarUserFinder):
        self.user_id = user_id
        self.user_item_matrix = user_item_matrix
        self.similar_user_finder = similar_user_finder

    def recommend(self, top_n: int = 5) -> List[int]:
        # Get similar user indices
        similar_user_indices = self.similar_user_finder.similar_users

        # Get item interactions of those users
        similar_users_matrix = self.user_item_matrix.iloc[similar_user_indices]

        # Sum interactions across similar users
        item_scores = similar_users_matrix.sum(axis=0)

        # Get items current user has already interacted with
        user_interactions = self.user_item_matrix.loc[self.user_id]

        # Filter out items the user already interacted with
        unseen_items = item_scores[user_interactions == 0]

        # Sort and return top N recommendations
        top_items = unseen_items.sort_values(ascending=False).head(top_n)

        return top_items.index.tolist()


## Item based recommendation system  

In [19]:
class IItemSimilarityFinder(ABC):
    @abstractmethod
    def find_similar_items(self, n_neighbors: int = 5) -> np.ndarray:
      """Finds and returns similar items to the given item ID."""
      pass

class ItemSimilarityFinder(IItemSimilarityFinder):
    def __init__(self, item_id: int, item_user_matrix: pd.DataFrame, knn_model: NearestNeighbors):
        self.item_user_matrix = item_user_matrix
        self.knn_model = knn_model
        self.item_id = item_id
        self._similar_videos = None

    def find_similar_items(self, n_neighbors: int = 5) -> np.ndarray:
        if self.item_id not in self.item_user_matrix.index:
            raise InvalidVideoIdError(self.item_id)

        item_vector = self.item_user_matrix.loc[self.item_id].values.reshape(1, -1)
        distances, indices = self.knn_model.kneighbors(item_vector, n_neighbors=n_neighbors + 1)

        return self.item_user_matrix.index[indices[0][1:]]

    @property
    def similar_videos(self) -> np.ndarray:
        if self._similar_videos is None:
            self._similar_videos = self.find_similar_items()
        return self._similar_videos


In [20]:
class IUserInteractionFetcher(ABC):
    @abstractmethod
    def get_interacted_items(self, user_id: int) -> List[int]:
        pass

class UserInteractionFetcher(IUserInteractionFetcher):
    def __init__(self, user_item_matrix: pd.DataFrame, user_checker: IUserInteractionChecker):
        self.user_item_matrix = user_item_matrix
        self.user_checker = user_checker

    def get_interacted_items(self, user_id: int) -> List[int]:
        self.user_checker.check_user_interaction(user_id, self.user_item_matrix)

        user_interactions = self.user_item_matrix.loc[user_id]
        return user_interactions[user_interactions > 0].index.tolist()

In [25]:
class IItemScoreAccumulator(ABC):
    @abstractmethod
    def reset(self) -> None:
        pass

    @abstractmethod
    def accumulate(self, similar_items: Iterable[int], interacted_items: Iterable[int]) -> None:
        pass

    @abstractmethod
    def get_scores(self) -> Dict[int, float]:
        pass

class ItemScoreAccumulator(IItemScoreAccumulator):
    def __init__(self):
        self._scores = {}

    def reset(self) -> None:
        self._scores.clear()

    def accumulate(self, similar_items: Iterable[int], interacted_items: Iterable[int]) -> None:
        for item in similar_items:
            if item not in interacted_items:
                self._scores[item] = self._scores.get(item, 0) + 1  # Simple count-based score

    def get_scores(self) -> Dict[int, float]:
        return self._scores


In [26]:
class IRecommendationRanker(ABC):
    @abstractmethod
    def rank(self, item_scores: Dict[int, float], top_n: int) -> List[int]:
        pass

class RecommendationRanker(IRecommendationRanker):
    def rank(self, item_scores: Dict[int, float], top_n: int) -> List[int]:
        sorted_items = sorted(item_scores.items(), key=lambda x: x[1], reverse=True)
        return [item for item, _ in sorted_items[:top_n]]

In [27]:
class ItemBasedRecommender:
    def __init__(self, user_id: int, user_item_matrix: pd.DataFrame,
                 similarity_finder: IItemSimilarityFinder, score_accumulator: IItemScoreAccumulator,
                 interaction_fetcher: IUserInteractionFetcher, ranker: IRecommendationRanker):
        self.user_id = user_id
        self.user_item_matrix = user_item_matrix
        self.similarity_finder = similarity_finder
        self.score_accumulator = score_accumulator
        self.interaction_fetcher = interaction_fetcher
        self.ranker = ranker

    def recommend(self, top_n: int = 5) -> List[int]:
        if self.user_id not in self.user_item_matrix.index:
            raise ValueError(f"User ID {self.user_id} not found in matrix.")

        # Fetch items the user has interacted with using UserInteractionFetcher
        interacted_items = self.interaction_fetcher.get_interacted_items(self.user_id)

        # Reset the scores accumulator
        self.score_accumulator.reset()

        # For each item the user interacted with, find similar items and accumulate scores
        for item in interacted_items:
            similar_items = self.similarity_finder.find_similar_items(item)
            self.score_accumulator.accumulate(similar_items, interacted_items)

        # Get the accumulated scores and use the ranker to rank the items
        item_scores = self.score_accumulator.get_scores()
        ranked_items = self.ranker.rank(item_scores, top_n)

        return ranked_items

In [37]:
def main():
    Dataset_path = "/content/modified_video_recommendation_dataset.csv"
    df = DataPreparingManager(Load_and_shuffle_Dataset, Dataset_path, InteractionScorer)
    df = df.prepare_data()

    user_item_builder = UserItemMatrix(df, row="user_id", column="video_id", values="interaction_score")
    user_item_matrix = user_item_builder.matrix
    item_user_matrix = user_item_matrix.T

    user_sparse_converter = SparseMatrixConverter(user_item_matrix)
    user_sparse_matrix = user_sparse_converter.sparse_matrix
    item_sparse_converter = SparseMatrixConverter(item_user_matrix)
    item_sparse_matrix = item_sparse_converter.sparse_matrix

    user_knn_trainer = KNNModelTrainer(sparse_matrix=user_sparse_matrix, metric='cosine', algorithm='brute')
    user_knn_model = user_knn_trainer.model
    item_knn_trainer = KNNModelTrainer(sparse_matrix=item_sparse_matrix, metric='cosine', algorithm='brute')
    item_knn_model = item_knn_trainer.model

    target_user_id = 1  # Replace with dynamic input if needed

    # ---- User-Based Recommendation ----
    user_checker = UserInteractionChecker()
    user_index_finder = UserIndexFinder(user_id=target_user_id,
                                        user_item_matrix=user_item_matrix,
                                        user_check=user_checker)

    similar_user_finder = SimilarUserFinder(user_id=target_user_id,
                                            knn_model=user_knn_model,
                                            user_item_matrix=user_item_matrix,
                                            user_index=user_index_finder,
                                            user_check=user_checker)

    user_recommender = UserBasedRecommender(user_id=target_user_id,
                                            user_item_matrix=user_item_matrix,
                                            similar_user_finder=similar_user_finder)

    user_recommendations = user_recommender.recommend(top_n=3)
    print(f"User-Based Recommendations for User {target_user_id}: {user_recommendations}")

    # ---- Item-Based Recommendation using ItemBasedRecommender ----
    # Dependencies
    interaction_checker = UserInteractionChecker()
    interaction_fetcher = UserInteractionFetcher(user_item_matrix=user_item_matrix,
                                                 user_checker=interaction_checker)
    score_accumulator = ItemScoreAccumulator()
    ranker = RecommendationRanker()

    # We'll pass the entire item-user matrix and model to a custom similarity finder adapter
    class ItemSimilarityAdapter(IItemSimilarityFinder):
        def __init__(self, item_user_matrix, knn_model):
            self.item_user_matrix = item_user_matrix
            self.knn_model = knn_model

        def find_similar_items(self, item_id: int, n_neighbors: int = 5) -> List[int]:
            if item_id not in self.item_user_matrix.index:
                raise InvalidVideoIdError(item_id)

            item_vector = self.item_user_matrix.loc[item_id].values.reshape(1, -1)
            distances, indices = self.knn_model.kneighbors(item_vector, n_neighbors=n_neighbors + 1)
            return self.item_user_matrix.index[indices[0][1:]].tolist()

    similarity_finder = ItemSimilarityAdapter(item_user_matrix, item_knn_model)

    item_recommender = ItemBasedRecommender(user_id=target_user_id,
                                            user_item_matrix=user_item_matrix,
                                            similarity_finder=similarity_finder,
                                            score_accumulator=score_accumulator,
                                            interaction_fetcher=interaction_fetcher,
                                            ranker=ranker)

    item_recommendations = item_recommender.recommend(top_n=3)
    print(f"Item-Based Recommendations for User {target_user_id}: {item_recommendations}")

if __name__ == "__main__":
    main()


User-Based Recommendations for User 1: [644, 3694, 1312]
Item-Based Recommendations for User 1: [3317, 2179, 754]


# **Content Filter**

# Recommendation System Error Exceptions

In [ ]:
class RecommendationSystemError(Exception):
    """Base class for all exceptions in the recommendation system."""
    pass

class InvalidMatrixError(RecommendationSystemError):
    """Raised when a matrix is invalid or does not meet the required conditions."""

    def __init__(self, message="Matrix is invalid"):
        self.message = message
        super().__init__(self.message)

class InvalidUserIdError(RecommendationSystemError):
    """Raised when a user ID is not found in the system."""

    def __init__(self, user_id, message="User ID not found"):
        self.user_id = user_id
        self.message = f"{message}: {user_id}"
        super().__init__(self.message)

class TrainingError(RecommendationSystemError):
    """Raised when an error occurs during model training."""

    def __init__(self, message="Model training failed"):
        self.message = message
        super().__init__(self.message)

class InvalidVideoIdError(RecommendationSystemError):
  """Raised when a video ID is not found in the system."""

  def __init__(self, video_id, message="Video ID not found"):
      self.video_id = video_id
      self.message = f"{message}: {video_id}"
      super().__init__(self.message)